In [1]:
"theatre|cafe|nightclub|kindergarten|doctors|fuel|bank|library|cinema|restaurant|atm|bar|fast_food|pharmacy|veterinary|taxi|brothel|university|police|events_venue|college|car_rental|clinic|community_centre|courier|food_court|social_facility|parking_space|hospital|waste_disposal|parcel_locker|charging_station|coworking_space|meeting_point|motorcycle_parking|childcare|social_centre|music_venue|healthcare|waste_transfer_station|casino|fire_station|student_accommodation|retail|prison|nursing_home|events_centre|exhibition_centre|conference_centre|biergarten|bus_station",


('theatre|cafe|nightclub|kindergarten|doctors|fuel|bank|library|cinema|restaurant|atm|bar|fast_food|pharmacy|veterinary|taxi|brothel|university|police|events_venue|college|car_rental|clinic|community_centre|courier|food_court|social_facility|parking_space|hospital|waste_disposal|parcel_locker|charging_station|coworking_space|meeting_point|motorcycle_parking|childcare|social_centre|music_venue|healthcare|waste_transfer_station|casino|fire_station|student_accommodation|retail|prison|nursing_home|events_centre|exhibition_centre|conference_centre|biergarten|bus_station',)

work: coworking_space, meeting_point, conference_centre, 
bars/restaurants:cafe, nightclub, restaurant, bar, fast_food, brothel, food_court, biergarten
greenspace: land area of suburb
cost of living: affordability into accnt (People want to make the most of lower housing cost options, while still having secure working opportunities)
median rental house price?
education: best school score in the isochrone and also count of schools and universities with tags; kindergarten, library, university, college, student_accommodation, 
health services: doctors, pharmacy, veterinary, clinic, hospital, healthcare, 
proximity to transport stops: 
safety: police, prison(negative affecting)
entertainment/social: not as much weighted; theatre, cinema, events_venue, social_facility, community_centre,  social_centre, music_venue, casino, retail, events_centre, exhibition_centre
service: fuel, bank, atm, taxi, car_rental, courier, parking_space, waste_disposal, parcel_locker, charging_station, motorcycle_parking, childcare, waste_transfer_station(negative affecting), fire_station, bus_station

Livability Equation
(L = 0.18,\hat{W} + 0.12,\hat{E} + 0.10,\hat{G} + 0.12,\hat{C} + 0.10,\hat{Ed} + 0.12,\hat{H} + 0.08,\hat{T} + 0.10,\hat{S} + 0.08,\hat{Sv} - 0.10,\hat{N})

Positive pillars use min–max scaled scores (\hat{x} \in [0,1]) for the isochrone around each suburb:
W: work density (coworking_space, meeting_point, conference_centre, etc.)
E: food/nightlife (cafe, nightclub, restaurant, bar, fast_food, brothel, food_court, biergarten)
G: greenspace share (e.g., % public green land)
C: affordability (inverse of median rent/price)
Ed: education access (schools, universities, libraries, student_accommodation)
H: health services (doctors, pharmacy, veterinary, clinic, hospital, healthcare)
T: transport stops within reach (bus_station, taxi, car_rental, etc.)
S: safety presence (police vs. crime proxies)
Sv: everyday services (fuel, bank, atm, parcel_locker, charging_station, etc.)
Negative pillar (\hat{N}) captures detractors (waste_transfer_station, prison, heavy_waste facilities). Count or area is scaled the same way and subtracted.
How to Read It
The weights follow your earlier intuition: work, affordability, and health matter most (0.12–0.18 each); education, safety, and food/nightlife are mid-tier (0.10–0.12); transport and services keep convenience in view (0.08); detractors get a negative weight to reduce the final index when undesirable features dominate. Because every subscore is normalized, (L) lives on roughly ([-0.10, 1.0]), making comparisons straightforward.

Suggested Next Steps

Re-check weight choices with team/user research or sensitivity analysis.
Confirm each subscore’s normalization window (city-wide vs. suburb-only).
Add columns to notebooks/liveability.ipynb calculating these pillars and the final (L); visualize quartiles or top suburbs once ready.